# QML frameworks

There are two most popular frameworks for quantum machine learning: pennylane (Bergholm et al. 2022) by Xanadu and TensorFlow Quantum (Broughton et al. 2021) from Google. There is also popular qiskit (Aleksandrowicz et al. 2019) framework from IBM, designed more for general quantum computing than for quantum machine learning, but it can also be used for ML (see for example [qiskit QML tutorials](https://qiskit.org/ecosystem/machine-learning/tutorials/index.html)).

In general these frameworks provide:
- interface for defining quantum circuits
- classical simulators of quantum circuits for easy small scale noiseless experiments
- interface for running circuits on real quantum hardware by different providers
- interface for seamless integration with classical machine learning frameworks (Pytorch and Tensorflow), which allow:
  - integration of quantum models to existing ML pipelines
  - constructing hybrid quantum/classical model


Except for fast circuit result prediction, ML tasks usually require efficient ways of gradients estimating. There are several popular derivative counting algorithms suitable for simulation or for real quantum hardware

### Finite difference
This is the simples default technic available both in simulation and on hardware. Works pretty bed with sampling based methods due to significant noise.

Can be expressed as:


$$\nabla_{\theta_i}\mathcal{M}_{\theta_i}(\hat{B}) = \frac{[\mathcal{M}_{\theta_i + d\theta}(\hat{B}) - \mathcal{M}_{\theta_i}(\hat{B})]}{d\theta}$$

or as

$$\frac{\partial}{\partial \alpha} f(\alpha) = \frac{1}{d\alpha} \cdot \left( f\left(\alpha + d\alpha\right) -  f\left(\alpha \right) \right)  $$

### Parameter shift
Parameter shifting rule is much more noise resistent technic compatible with sampling. 
It was first introduced to quantum machine learning in [Mitarai et al. (2018)](https://arxiv.org/abs/1803.00745 "‌"), and extended in [Schuld et al. (2018)](https://arxiv.org/abs/1811.11184 "‌").

In these works it was proved, that for any one qubit gate such a wonder full property is fulfilled. For these gates exist finite constants $с$ and $s$ that

$$\nabla_{\theta_i}\mathcal{M}_{\theta_i}(\hat{B}) = c[\mathcal{M}_{\theta_i + s}(\hat{B}) - \mathcal{M}_{\theta_i - s}(\hat{B})],$$
which means that infinitely small difference can be replaced with some specific big difference, but equality will still hold.

For example for PauliY gate \
$|\alpha⟩ = Y^{\alpha}|0⟩$ \
$f(\alpha) = ⟨\alpha|X|\alpha⟩$

$$\frac{\partial}{\partial \alpha} f(\alpha) = \frac{\pi}{2} f\left(\alpha + \frac{1}{2}\right) -  \frac{ \pi}{2} f\left(\alpha - \frac{1}{2}\right)$$

## Simulation technics
Since simulators play important role in research process, tricks that improve algorithm running speed only in simulation were also developed, although they will loose their significance in post NISQ era.

### Adjoint method
Detailed tutorial with theory explanations can be found in [demo](https://pennylane.ai/qml/demos/tutorial_adjoint_diff.html) from pennyLane.

But in a nutshell this method accelerate gradient evaluation by exploiting reversibility of quantum operations. While inverse matrix computation in general is relatively slow, for unitary matrix representing quantum operations by definition inverse equals to adjoint (transposition + complex conjugate) with is almost computationally free.

Considering this, algorithm is as follows:

$$\langle M \rangle = \langle b | k \rangle = \langle \Psi | M | \Psi \rangle$$

Firstly decompose the expectation into the scalar product of two vectors

$$\langle b | = \langle \Psi| M = \langle 0 | U_0^{\dagger} \dots U_n^{\dagger} M$$
$$| k \rangle =  |\Psi \rangle = U_n U_{n-1} \dots U_0 |0\rangle$$

change the location of the partition by transferring the matrices in turn from one vector to another.

$$\langle b_i | = \langle 0 | U_1^{\dagger} \dots U_n^{\dagger} M U_n \dots U_{i+1}$$
$$|k_i \rangle = U_{i-1} \dots U_1 |0\rangle$$

$$\langle b_i | = \langle b_{i+1}| U_{i}$$
$$|k_{i} \rangle = U_{i+1}^{\dagger} |k_{i+1}\rangle$$

To calculate the derivative, the matrix derivative with respect to the parameter is inserted into the required partition.

$$\langle 0 | U_1^{\dagger} \dots U_i^{\dagger} \dots M \dots \frac{\text{d} U_i}{\text{d} \theta_i}  \dots U_1 |0\rangle$$
$$= \langle b_i | \frac{\text{d} U_i}{\text{d} \theta_i} |k_i \rangle$$


<img src="https://pennylane.ai/_images/scaling.png" width=800>\
benchmark results from pennylane demo

Both pennylane and tf quantum provide fast adjoint simulation and the overall performance in best configuration is almost the same. This is demonstrated by our tests (see plot below). Source code for benchmarks reproducing is located in `qml_frameworks_benchmarking`

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/benchmarks.png" width=400>\
frameworks speed comparison.

## Closing remark
These frameworks are also able to simulate realistic noise, they are compatible with wide range of quantum hardware providers (see table below for comparison)


| **Company**                 | **TensorFlow(cirq) support** | **PennyLane Support** | **# qubits** |
|-----------------------------|------------------------------|-----------------------|--------------|
| IBM                         | - ([link 2020](https://quantumcomputing.stackexchange.com/questions/14164/can-i-run-cirq-on-ibmq))| +                     | 127(up to 7 free)|
| Amazon Braket               |                              | +                     |              |
| Google(Cirq)                | +                            | +                     | 53           |
| Microsoft                   | +                            | +                     | ?            |
| Honeywell (Quantinuum)      |                              | +                     | 20           |
| IonQ                        | +                            | +                     | 34           |
| Alpine Quantum Technologies | +                            | +                     |              |
| Rigetti                     | +                            | +                     | 80           |
| Pasqal                      | +                            | +(through cirq)       | 100          |
| Xanadu                      |                              | +                     | 200+         |

Although these frameworks are pretty similar,  there is one notable difference. The hallmark of Pennylane is the support of Continuous Variable quantum computing, which is a topic of the next section. 


## Reference

Aleksandrowicz, Gadi, Thomas Alexander, Panagiotis Barkoutsos, Luciano Bello, Yael Ben-Haim, David Bucher, Francisco Jose Cabrera-Hernández, et al. 2019. “Qiskit: An Open-Source Framework for Quantum Computing.” Zenodo. https://doi.org/10.5281/zenodo.2562111.

Bergholm, Ville, Josh Izaac, Maria Schuld, Christian Gogolin, Shahnawaz Ahmed, Vishnu Ajith, M. Sohaib Alam, et al. 2022. “PennyLane: Automatic Differentiation of Hybrid Quantum-Classical Computations.” arXiv. https://doi.org/10.48550/arXiv.1811.04968.

Broughton, Michael, Guillaume Verdon, Trevor McCourt, Antonio J. Martinez, Jae Hyeon Yoo, Sergei V. Isakov, Philip Massey, et al. 2021. “TensorFlow Quantum: A Software Framework for Quantum Machine Learning.” arXiv. https://doi.org/10.48550/arXiv.2003.02989.
